# Course 2: Project - Task E - Build a database

<a name="task-e-top"></a>
This notebook is concerned with task E.

**Contents:**
* [Imports](#task-e-imports)
* [Utilities](#task-e-utilities)
* [Data loading](#task-e-data-loading)
* [Build the database](#task-e-build-the-database)

## Imports<a name="task-e-imports"></a> ([top](#task-e-top))
---

In [75]:
# Standard library:
import pathlib
import sqlite3
import typing as t

# 3rd party:
import pandas as pd

# Project:
import utils

## Utilities<a name="task-e-utilities"></a> ([top](#task-e-top))
---

We define a few utilities that we will use later in this notebook:

In [56]:
QueryParams = t.Optional[t.Union[t.Sequence[t.Any], t.Mapping[str, t.Any]]]

def pd_execute(con: sqlite3.Connection, query: str, params: QueryParams = None) -> pd.DataFrame:
    return pd.read_sql_query(query, con, params=params)


def pd_explain(con: sqlite3.Connection, query: str, params: QueryParams = None) -> pd.DataFrame:
    return pd.read_sql_query('EXPLAIN QUERY PLAN ' + query, con, params=params)


def execute(cursor: sqlite3.Cursor, query: str, params: QueryParams = None) -> sqlite3.Cursor:
    if params:
        cursor.execute(query, params)
    else:
        cursor.execute(query)
    return cursor


def fetchall(cursor: sqlite3.Cursor, query: str, params: QueryParams = None) -> t.List[t.Any]:
    return execute(cursor, query, params).fetchall()


def fetchone(cursor: sqlite3.Cursor, query: str, params: QueryParams = None) -> t.Optional[t.Any]:
    return execute(cursor, query, params).fetchone()

## Data loading<a name="task-e-data-loading"></a> ([top](#task-e-top))
---

First, we load the subset of the cleaned-up dataset that we need:

In [3]:
base_name = pathlib.Path.cwd().joinpath('en.openfoodfacts.org.products.clean')

As indicated, we restrict the data to 1000 rows and 5 columns of our choice:

In [72]:
# The columns to load:
usecols=[
    'code',
    'url',
    'creator',
    'created_on_utc',
    'product_name'
]

# Load the data-types:
data_types, parse_dates = utils.load_and_amend_dtypes(base_name)
# We can only parse dates in the columns that we are loading:
parse_dates = list(set(parse_dates) & set(usecols))

# Load the data-frame:
df = pd.read_csv(
    f'{base_name}.csv',
    header=0,
    parse_dates=parse_dates,
    usecols=usecols,
    nrows=1000,
    dtype=data_types)

We get some general information:

In [44]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
code              1000 non-null object
url               1000 non-null object
creator           1000 non-null object
product_name      933 non-null object
created_on_utc    1000 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 39.1+ KB


We look at the first few rows:

In [45]:
df.head()

,code,url,creator,product_name,created_on_utc
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,Farine de blé noir,2016-09-17 09:17:46
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,Banana Chips Sweetened (Whole),2017-03-09 14:32:37
2,0000000004559,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,Peanuts,2017-03-09 14:32:37
3,0000000016087,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,Organic Salted Nut Mix,2017-03-09 10:35:31
4,0000000016094,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,Organic Polenta,2017-03-09 10:34:13


## Build the database<a name="task-e-build-the-database"></a> ([top](#task-e-top))
---

We decide to call the databse *test* and to store it in the same directory as this notebook:

In [48]:
db_name = pathlib.Path.cwd().joinpath('test.db')

### Create a connection

In [50]:
con = sqlite3.connect(str(db_name))

Given the time spent on the other tasks, we implement the basis requirements for this task and create only 1 table. We confirm that `code` is suitable as a PRIMARY KEY:

In [11]:
df['code'].is_unique

True

### Create one table

We create the `test` table. Since SQLite has a limited set of data-types, we decide to store [dates and times](https://www.sqlite.org/datatype3.html#date_and_time_datatype) as ISO 8601 strings.

In [57]:
query_drop = '''\
DROP TABLE IF EXISTS test
'''

query_create = '''\
CREATE TABLE IF NOT EXISTS test (
    code TEXT PRIMARY KEY,
    url TEXT,
    creator TEXT,
    created_on_utc TEXT,
    product_name TEXT
);
'''

with con:
    cursor = con.cursor()
    execute(cursor, query_drop)
    execute(cursor, query_create)

We check the result:

In [66]:
pd_execute(con, "PRAGMA table_info('test');")

,cid,name,type,notnull,dflt_value,pk
0,0,code,TEXT,0,None,1
1,1,url,TEXT,0,None,0
2,2,creator,TEXT,0,None,0
3,3,created_on_utc,TEXT,0,None,0
4,4,product_name,TEXT,0,None,0


### Fill the database

We fill the database with our data:

In [67]:
df.to_sql(name='test', con=con, if_exists='append', index=False)

We check the result:

In [69]:
pd_execute(con, 'SELECT * FROM test LIMIT 5;')

,code,url,creator,created_on_utc,product_name
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2016-09-17 09:17:46,Farine de blé noir
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09 14:32:37,Banana Chips Sweetened (Whole)
2,0000000004559,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09 14:32:37,Peanuts
3,0000000016087,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09 10:35:31,Organic Salted Nut Mix
4,0000000016094,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,2017-03-09 10:34:13,Organic Polenta


### Run one query

We select all entries created in January 2017:

In [70]:
query = '''\
SELECT *
FROM test
WHERE strftime('%Y-%m-%d', created_on_utc) BETWEEN '2017-01-01' AND '2017-01-31';
'''

pd_execute(con, query)

,code,url,creator,created_on_utc,product_name
0,0000000290616,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-16 22:32:58,Salade Cesar
1,0000000394710,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-15 16:22:50,Danoises à la cannelle roulées
2,0000001938067,http://world-en.openfoodfacts.org/product/0000...,b7,2017-01-15 17:32:08,Chaussons tressés aux pommes
3,0000013000004,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,2017-01-14 22:13:19,Kochspeck
4,0000069016165,http://world-en.openfoodfacts.org/product/0000...,date-limite-app,2017-01-27 17:52:15,BoFrost Nudelpfanne
5,0000250632969,http://world-en.openfoodfacts.org/product/0000...,allfitnessfactory-de,2017-01-13 07:30:12,Mehrkomponeneten Protein 90 C6 Banane
6,0000470322800,http://world-en.openfoodfacts.org/product/0000...,allfitnessfactory-de,2017-01-13 10:22:12,Whey Protein aus Molke Vanilla
7,0000501050603,http://world-en.openfoodfacts.org/product/0000...,allfitnessfactory-de,2017-01-13 10:12:31,Whey Protein aus Molke 1000 Gramm Vanilla
8,0000526938306,http://world-en.openfoodfacts.org/product/0000...,allfitnessfactory-de,2017-01-13 10:03:47,Whey Protein aus Molke 500 Gramm Vanilla
9,0001000008706,http://world-en.openfoodfacts.org/product/0001...,date-limite-app,2017-01-14 15:07:01,Huile d'olive Monini


### Cleanup

Finally, we close the connection:

In [74]:
con.close()